In [1]:
import numba as nb
import numpy as np

In [93]:
@nb.jit(nb.float64[:](nb.float64[:], nb.float64[:]), nopython=True)
def cross_product(a, b):

    output=np.empty((3), dtype=float)

    output[0]=a[1]*b[2]-a[2]*b[1]
    output[1]=-a[0]*b[2]+a[2]*b[0]
    output[2]=a[0]*b[1]-a[1]*b[0]

    return output

In [95]:
aa = np.array([0,1,2,3])

In [96]:
a,b,c,d = aa

In [97]:
a

0

In [79]:
aa = [[3,4,5],[1,10],[3,4,6,7],[8],[2,9,1]]

In [80]:
bb = nb.typed.List([nb.typed.List(ii) for ii in aa])

In [81]:
nb.jit(nb.types.UniTuple(nb.int64[:], 2)(nb.types.List(dtype=nb.types.List(dtype=nb.int64))), nopython=True)
def test(item):
    
    n_values=0
    n_starts=0
    for segment in item:
        n_values+=len(segment)
        n_starts+=1
    
    values = np.empty((n_values), dtype=int)
    starts = np.empty((n_starts+1), dtype=int)
    
    counter=0
    for ii,segment in enumerate(item):
        starts[ii]=counter
        for jj in sorted(segment):
            values[counter]=jj
            counter+=1
    
    starts[-1]=counter
    
    return starts, values

In [83]:
starts, values = test(bb)

In [84]:
n = starts.shape[0]-1

In [87]:
for ii in range(n):
    print('xxxx')
    for jj in range(starts[ii],starts[ii+1]):
        print(values[jj])

xxxx
3
4
5
xxxx
1
10
xxxx
3
4
6
7
xxxx
8
xxxx
1
2
9


In [ ]:
class serialized_lists():

    def __init__ (self, item=None):

        self.values = None
        self.starts = None
        self.indices = None
        self.n_values = None
        self.n_indices = None

        if type(item) is list or type(item) is np.ndarray:

            """
            item = [[3,4,5],[1,10],[3,4,6,7],[8],[2,9,1]]
            serialized_item = serialized_lists(item)
            
            serialized_item.values = [3,4,5,1,10,3,4,6,7,8,1,2,9]
            serialized_item.starts = [0,3,5,9,10,13]
            serialized_item.indices = [0,1,2,3,4]
            serialized_item.n_values = 13
            serialized_item.n_indices = 5
            """

            self.values =  [np.sort(ll) for ll in item]
            self.values = np.concatenate(self.values)
            self.indices = np.arange(len(item))
            self.starts= []
            jj = 0
            for ii in item:
                self.starts.append(jj)
                jj+=len(ii)
            self.starts.append(jj)

            self.starts = np.array(self.starts, dtype=int)

            self.n_values = self.values.shape[0]
            self.n_indices = self.indices.shape[0]

        elif type(item) is dict:

            """
            item = {2:[3,4,5],7:[1,10],8:[3,4,6,7],3:[8],6:[2,9,1]}
            serialized_item = serialized_lists(item)
            
            item = [2:[3,4,5],3:[8],6:[2,9,1],7:[1,10],8:[3,4,6,7]]
            serialized_item.values = [3,4,5,8,1,2,9,1,10,3,4,6,7]
            serialized_item.starts = [0,3,4,7,9,13]
            serialized_item.indices = [2,3,6,7,8]
            serialized_item.n_values = 13
            serialized_item.n_indices = 5
            """

            self.indices = np.sort(list(item.keys()))

            self.values = []
            self.starts= []
            jj = 0
            for ii in self.indices:
                self.values.append(np.sort(item[ii]))
                self.starts.append(jj)
                jj+=len(item[ii])
            self.starts.append(jj)

            self.values = np.concatenate(self.values)
            self.starts = np.array(self.starts, dtype=int)

            self.n_values = self.values.shape[0]
            self.n_indices = self.indices.shape[0]
